Implemented hit selection, events scatter, ion heatmap, electrons time of flight, ions time of flight, merging multiple runs, spatial selection view, calibration choice of lines with corresponding zoomed-in views, calibration curve fit and apply with mq column

Additions:
+ make multiple threshold selections
+ spatial selection view with angle
+ fish plots
+ binning by number of events
+ choose appropriate binning for mq and ion tof
+ background with multiple runs downsampled to selected number of pulses
+ waterfall plots
+ heatmaps for binned number of events

# Imports and functions

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter